In [29]:
! uv pip install python-docx
! uv pip install pdfplumber
! uv pip install PyPDF2
! uv pip install olefile
! uv pip install PyMuPDF
! uv pip install markdown-it-py
! uv pip install markdown

Using Python 3.12.6 environment at: /Users/young/project/llmProject/deep_research_from_scratch/.venv
Audited 1 package in 17ms
Using Python 3.12.6 environment at: /Users/young/project/llmProject/deep_research_from_scratch/.venv
Audited 1 package in 7ms
Using Python 3.12.6 environment at: /Users/young/project/llmProject/deep_research_from_scratch/.venv
Audited 1 package in 3ms
Using Python 3.12.6 environment at: /Users/young/project/llmProject/deep_research_from_scratch/.venv
Audited 1 package in 3ms
Using Python 3.12.6 environment at: /Users/young/project/llmProject/deep_research_from_scratch/.venv
Audited 1 package in 3ms
Using Python 3.12.6 environment at: /Users/young/project/llmProject/deep_research_from_scratch/.venv
Audited 1 package in 3ms
Using Python 3.12.6 environment at: /Users/young/project/llmProject/deep_research_from_scratch/.venv
Resolved 1 package in 1.08s                                          
⠙ Preparing packages... (0/1)                                           

In [24]:
from pathlib import Path
import os

def get_current_dir() -> Path:
    """获取正确的文件目录"""
    notebook_dir = Path.cwd()
    return notebook_dir / "files"

def convert_doc_to_docx(file_path):
    """将.doc文件转换为.docx格式"""
    try:
        import subprocess
        import platform
        
        docx_path = file_path.with_suffix('.docx')
        
        if platform.system() == "Darwin":  # macOS
            # 使用 textutil (macOS内置工具)
            subprocess.run([
                'textutil', '-convert', 'docx', 
                '-output', str(docx_path), str(file_path)
            ], check=True, capture_output=True)
        elif platform.system() == "Windows":
            # Windows可以使用 win32com 或其他工具
            # 这里简化处理，建议先手动转换
            print(f"请手动将 {file_path.name} 转换为 .docx 格式")
            return None
        else:
            # Linux可以使用 libreoffice
            subprocess.run([
                'libreoffice', '--headless', '--convert-to', 'docx',
                '--outdir', str(file_path.parent), str(file_path)
            ], check=True, capture_output=True)
            
        return docx_path
    except Exception as e:
        print(f"转换失败 {file_path.name}: {e}")
        return None

def extract_text_from_doc(file_path):
    """使用olefile提取.doc文件中的文本"""
    try:
        import olefile
        
        if not olefile.isOleFile(file_path):
            return None
            
        ole = olefile.OleFileIO(file_path)
        
        # 尝试不同的流来提取文本
        text_streams = ['WordDocument', 'Body', 'Text']
        text_content = []
        
        for stream_name in text_streams:
            if ole.exists(stream_name):
                stream = ole.openstream(stream_name)
                data = stream.read()
                
                # 简单的文本提取 (这不是完美的解决方案)
                try:
                    text = data.decode('utf-8', errors='ignore')
                    if text.strip():
                        text_content.append(text)
                except:
                    continue
        
        ole.close()
        
        if text_content:
            return '\n'.join(text_content)
        else:
            return None
            
    except Exception as e:
        print(f"olefile提取失败 {file_path.name}: {e}")
        return None

def convert_files_to_text():
    """改进版的文档转换函数，支持.doc格式"""
    files_dir = get_current_dir()
    converted_dir = files_dir.parent / "converted"
    converted_dir.mkdir(exist_ok=True)
    
    conversions = []
    
    for file_path in files_dir.glob("**/*"):
        if not file_path.is_file():
            continue
            
        file_ext = file_path.suffix.lower()
        file_name = file_path.stem
        
        try:
            if file_ext == '.docx':
                # 处理 .docx 文件
                from docx import Document
                doc = Document(file_path)
                text_content = [p.text for p in doc.paragraphs if p.text.strip()]
                
                txt_path = converted_dir / f"{file_name}.txt"
                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write('\n\n'.join(text_content))
                
                conversions.append(f"✓ {file_path.name} -> {txt_path.name}")
                
            elif file_ext == '.doc':
                # 处理 .doc 文件 - 方法1: 尝试转换为docx
                print(f"正在处理 .doc 文件: {file_path.name}")
                
                # 方法1: 尝试自动转换
                docx_path = convert_doc_to_docx(file_path)
                if docx_path and docx_path.exists():
                    from docx import Document
                    doc = Document(docx_path)
                    text_content = [p.text for p in doc.paragraphs if p.text.strip()]
                    
                    txt_path = converted_dir / f"{file_name}.txt"
                    with open(txt_path, 'w', encoding='utf-8') as f:
                        f.write('\n\n'.join(text_content))
                    
                    conversions.append(f"✓ {file_path.name} -> {txt_path.name} (已转换)")
                    
                    # 清理临时文件
                    docx_path.unlink()
                else:
                    # 方法2: 使用olefile直接提取
                    text_content = extract_text_from_doc(file_path)
                    if text_content:
                        txt_path = converted_dir / f"{file_name}.txt"
                        with open(txt_path, 'w', encoding='utf-8') as f:
                            f.write(text_content)
                        
                        conversions.append(f"✓ {file_path.name} -> {txt_path.name} (olefile提取)")
                    else:
                        conversions.append(f"⚠ {file_path.name}: 无法提取文本，请手动转换为.docx格式")
                
            elif file_ext == '.pdf':
                # 处理 PDF 文件
                import pdfplumber
                text_content = []
                
                with pdfplumber.open(file_path) as pdf:
                    for page in pdf.pages:
                        page_text = page.extract_text()
                        if page_text:
                            text_content.append(page_text)
                
                txt_path = converted_dir / f"{file_name}.txt"
                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write('\n\n'.join(text_content))
                
                conversions.append(f"✓ {file_path.name} -> {txt_path.name}")
                
            else:
                conversions.append(f"⚠ {file_path.name}: 不支持的文件格式 {file_ext}")
                
        except Exception as e:
            conversions.append(f"✗ {file_path.name}: {str(e)}")
    
    return "\n".join(conversions)

# 检查依赖
def check_dependencies():
    """检查所需的依赖包"""
    required_packages = {
        'docx': 'python-docx',
        'pdfplumber': 'pdfplumber',
        'olefile': 'olefile'
    }
    
    missing = []
    for module, package in required_packages.items():
        try:
            __import__(module)
            print(f"✓ {package} 已安装")
        except ImportError:
            missing.append(package)
            print(f"✗ {package} 未安装")
    
    if missing:
        print(f"\n请安装缺失的包: uv pip install {' '.join(missing)}")
        return False
    return True

In [25]:
# 执行转换
if check_dependencies():
    print("正在转换文档文件...")
    conversion_result = convert_files_to_text()
    print(conversion_result)
else:
    print("请先安装必要的依赖包")

✓ python-docx 已安装
✓ pdfplumber 已安装
✓ olefile 已安装
正在转换文档文件...
正在处理 .doc 文件: 赵庄矿项目-可行性研究报告初稿.doc
✓ 赵庄矿项目-可行性研究报告初稿.doc -> 赵庄矿项目-可行性研究报告初稿.txt (已转换)
✓ 塔山矿项目配套情况.docx -> 塔山矿项目配套情况.txt
✓ 附件2晋能控股煤业集团科技研发计划项目可行性研究报告.docx -> 附件2晋能控股煤业集团科技研发计划项目可行性研究报告.txt
✓ 晋能控股煤业集团有限公司关于安排征集2026年度研发计划的通知_1.pdf -> 晋能控股煤业集团有限公司关于安排征集2026年度研发计划的通知_1.txt


In [33]:
# 调试版本 - 让我们先看看路径和文件
def debug_paths():
    """调试路径和文件位置"""
    current_dir = Path.cwd()
    print(f"当前工作目录: {current_dir}")
    
    # 检查可能的文件目录
    possible_dirs = [
        current_dir / "files",
        current_dir / "my_src_mod" / "files"
    ]
    
    for directory in possible_dirs:
        print(f"\n检查目录: {directory}")
        if directory.exists():
            print(f"  ✓ 目录存在")
            files = list(directory.glob("*"))
            print(f"  文件数量: {len(files)}")
            for file in files:
                print(f"    - {file.name} ({file.suffix})")
        else:
            print(f"  ✗ 目录不存在")
    
    return possible_dirs

def get_correct_files_dir() -> Path:
    """获取正确的文件目录"""
    current_dir = Path.cwd()
    
    # 按优先级检查可能的目录
    possible_dirs = [
        current_dir / "deep_research_from_scratch" / "files",
        current_dir / "files",
        current_dir / "my_src_mod" / "deep_research_from_scratch" / "files"
    ]
    
    for directory in possible_dirs:
        if directory.exists() and any(directory.glob("*")):
            print(f"使用文件目录: {directory}")
            return directory
    
    # 如果都不存在，返回第一个作为默认
    print(f"警告: 没有找到包含文件的目录，使用默认: {possible_dirs[0]}")
    return possible_dirs[0]

def convert_files_to_markdown_fixed():
    """修复版本的文档转Markdown函数"""
    # 使用修复的目录查找函数
    files_dir = get_correct_files_dir()
    markdown_dir = files_dir.parent / "markdown"
    markdown_dir.mkdir(exist_ok=True)
    
    print(f"源文件目录: {files_dir}")
    print(f"输出目录: {markdown_dir}")
    
    conversions = []
    file_count = 0
    
    for file_path in files_dir.glob("**/*"):
        if not file_path.is_file():
            continue
            
        file_count += 1
        file_ext = file_path.suffix.lower()
        file_name = file_path.stem
        
        print(f"\n处理文件 {file_count}: {file_path.name} (类型: {file_ext})")
        
        try:
            markdown_content = ""
            
            if file_ext == '.docx':
                print("  -> 使用DOCX转换器")
                markdown_content = convert_docx_to_markdown(file_path)
                
            elif file_ext == '.doc':
                print("  -> 使用DOC转换器")
                # 先转换为docx，然后转为markdown
                docx_path = convert_doc_to_docx(file_path)
                if docx_path and docx_path.exists():
                    markdown_content = convert_docx_to_markdown(docx_path)
                    docx_path.unlink()  # 清理临时文件
                    print("  -> DOC转换成功")
                else:
                    conversions.append(f"⚠ {file_path.name}: 无法转换，请手动转换为.docx格式")
                    print("  -> DOC转换失败")
                    continue
                    
            elif file_ext == '.pdf':
                print("  -> 使用PDF转换器")
                markdown_content = convert_pdf_to_markdown(file_path)
                
            else:
                conversions.append(f"⚠ {file_path.name}: 不支持的文件格式 {file_ext}")
                print(f"  -> 不支持的格式: {file_ext}")
                continue
            
            if markdown_content and markdown_content.strip():
                md_path = markdown_dir / f"{file_name}.md"
                with open(md_path, 'w', encoding='utf-8') as f:
                    f.write(markdown_content)
                
                conversions.append(f"✓ {file_path.name} -> {md_path.name}")
                print(f"  -> 转换成功，输出到: {md_path.name}")
                print(f"  -> 内容长度: {len(markdown_content)} 字符")
            else:
                conversions.append(f"✗ {file_path.name}: 转换失败，内容为空")
                print("  -> 转换失败，内容为空")
                
        except Exception as e:
            error_msg = f"✗ {file_path.name}: {str(e)}"
            conversions.append(error_msg)
            print(f"  -> 错误: {e}")
    
    if file_count == 0:
        conversions.append("⚠ 没有找到任何文件进行转换")
    
    return "\n".join(conversions)

# 先运行调试
print("=== 调试路径信息 ===")
debug_paths()

print("\n=== 开始转换 ===")
conversion_result = convert_files_to_markdown_fixed()
print("\n=== 转换结果 ===")
print(conversion_result)

=== 调试路径信息 ===
当前工作目录: /Users/young/project/llmProject/deep_research_from_scratch/my_src_mod

检查目录: /Users/young/project/llmProject/deep_research_from_scratch/my_src_mod/files
  ✓ 目录存在
  文件数量: 4
    - 赵庄矿项目-可行性研究报告初稿.doc (.doc)
    - 塔山矿项目配套情况.docx (.docx)
    - 附件2晋能控股煤业集团科技研发计划项目可行性研究报告.docx (.docx)
    - 晋能控股煤业集团有限公司关于安排征集2026年度研发计划的通知_1.pdf (.pdf)

检查目录: /Users/young/project/llmProject/deep_research_from_scratch/my_src_mod/my_src_mod/files
  ✗ 目录不存在

=== 开始转换 ===
使用文件目录: /Users/young/project/llmProject/deep_research_from_scratch/my_src_mod/deep_research_from_scratch/files
源文件目录: /Users/young/project/llmProject/deep_research_from_scratch/my_src_mod/deep_research_from_scratch/files
输出目录: /Users/young/project/llmProject/deep_research_from_scratch/my_src_mod/deep_research_from_scratch/markdown

处理文件 1: 赵庄矿项目-可行性研究报告初稿.doc (类型: .doc)
  -> 使用DOC转换器
  -> DOC转换成功
  -> 转换成功，输出到: 赵庄矿项目-可行性研究报告初稿.md
  -> 内容长度: 6434 字符

处理文件 2: 塔山矿项目配套情况.docx (类型: .docx)
  -> 使用DOCX转换器
  -> 转换成功，输出到: 塔山矿项目配套情况

In [34]:
# 执行Markdown转换
print("正在转换文档为Markdown格式...")
conversion_result = convert_files_to_markdown()
print(conversion_result)

正在转换文档为Markdown格式...
正在处理 .doc 文件: 赵庄矿项目-可行性研究报告初稿.doc
✓ 赵庄矿项目-可行性研究报告初稿.doc -> 赵庄矿项目-可行性研究报告初稿.md
✓ 塔山矿项目配套情况.docx -> 塔山矿项目配套情况.md
✓ 附件2晋能控股煤业集团科技研发计划项目可行性研究报告.docx -> 附件2晋能控股煤业集团科技研发计划项目可行性研究报告.md
✓ 晋能控股煤业集团有限公司关于安排征集2026年度研发计划的通知_1.pdf -> 晋能控股煤业集团有限公司关于安排征集2026年度研发计划的通知_1.md
